# FacsimiLab for scRNAseq

This jupyter notebook is designed to test the FacsimiLab docker container's ability to analyze single-cell RNA sequencing (scRNAseq) data. It utilizes `scvi`, `scanpy`, and `pytorch`.


This jupyter notebook is a modification of the scverse tutorial called [Introduction to scvi-tools](https://docs.scvi-tools.org/en/stable/tutorials/notebooks/quick_start/api_overview.html). The original source code is available [on Github](https://github.com/scverse/scvi-tutorials/blob/c62f43f1c8c58710d99afe2e0d374c17a587b566/quick_start/api_overview.ipynb). We'd like to thank the YosefLab for their incredible tools and resources. This tutorial notebooks is licensed with **BSD 3-Clause License** and a complete copy of their license can be found at the end of this notebook.


In [ ]:
import os
import sys
import tempfile
from IPython.display import display, Markdown

import anndata
import pooch
import muon

import scanpy as sc
import scvi
import torch

from scipy.sparse import csr_matrix
import pandas as pd
import numpy as np

import jax
import jaxlib
import flax

In [ ]:
# Check if Pytorch has succssfully detected and loaded an Nvidia GPU with CUDA support
if torch.cuda.is_available():

    display(Markdown("## Facsimilab: Nvidia CUDA GPU Detected"))
    display(Markdown(f"GPU Name: {torch.cuda.get_device_name(0)}"))
    display(Markdown(f"GPU Available: {torch.cuda.is_available()}"))

    display(Markdown("### System Information"))

    display(
        Markdown(
            f"- Python version: `{sys.version}` \n - PyTorch version: `{torch.__version__}`\n - CUDNN version: `{torch.backends.cudnn.version()}`\n - Number CUDA Devices: `{torch.cuda.device_count()}`"
        )
    )

    display(Markdown("### Devices"))

    display(
        Markdown(
            f"- Available devices `{torch.cuda.device_count()}`\n - Active CUDA device: `{torch.cuda.current_device()}`"
        )
    )

    display(
        Markdown(
            "Python starts numbering from '0'. Therefore, the `Active CUDA device` name/number is expected to be `0` above."
        )
    )

else:
    display(Markdown("## No CUDA GPU Detected"))
    display(
        Markdown(
            "This notebook will use the CPU instead of the GPU. Analysis time is expected to be _**significantly longer, but still possible.**_"
        )
    )

    display(Markdown(f"GPU Available: {torch.cuda.is_available()}"))

    display(Markdown("### System Information"))

    display(
        Markdown(
            f"- Python version: `{sys.version}` \n - PyTorch version: `{torch.__version__}`\n - CUDNN version: `{torch.backends.cudnn.version()}`\n - Number CUDA Devices: `{torch.cuda.device_count()}`"
        )
    )

In [ ]:
scvi.settings.seed = 0
sc.set_figure_params(figsize=(6, 6))
torch.set_float32_matmul_precision("medium")
%config InlineBackend.print_figure_kwargs={'facecolor' : "w"}
%config InlineBackend.figure_format='retina'

## Loading and preparing data

Let us first load a subsampled version of the heart cell atlas dataset described in Litviňuková et al. (2020). scvi-tools has many "built-in" datasets as well as support for loading arbitrary `.csv`, `.loom`, and `.h5ad` (AnnData) files. Please see our tutorial on data loading for more examples.

-   Litviňuková, M., Talavera-López, C., Maatz, H., Reichart, D., Worth, C. L., Lindberg, E. L., ... & Teichmann, S. A. (2020). Cells of the adult human heart. Nature, 588(7838), 466-472.

```{important}
All scvi-tools models require AnnData objects as input.
```


In [ ]:
data_directory = "./data"
verbosity = True

In [ ]:
def download_data(
    save_path: str, fname: str = "pbmc5k_protein_filtered_feature_bc_matrix.h5"
) -> str:
    """Download the data files."""
    return pooch.retrieve(
        url="https://cf.10xgenomics.com/samples/cell-exp/3.0.2/5k_pbmc_protein_v3/5k_pbmc_protein_v3_filtered_feature_bc_matrix.h5",
        known_hash="7695e6b1888bdae6f53b3a28a99f0a0cdf387d1685e330a597cdd4b5541f8abd",
        fname=fname,
        path=save_path,
    )

In [ ]:
adata_heart = scvi.data.heart_cell_atlas_subsampled(save_path=data_directory)
adata_heart.write_h5ad(f"./data/heart_cell_atlas_supersubsampled.h5ad")
adata_heart

In [ ]:
pbmc3k_path = os.path.join(data_directory, "pbmc3k.h5ad")
pbmc3k = sc.read(
    filename=pbmc3k_path, backup_url="http://falexwolf.de/data/pbmc3k_raw.h5ad"
)
pbmc3k

In [ ]:
h5_path = download_data(data_directory)
pbmc5k = muon.read_10x_h5(h5_path)
pbmc5k.var_names_make_unique()
pbmc5k

In [ ]:
adata = anndata.concat([pbmc3k, pbmc5k.mod["rna"]], join="inner", label="batch")

In [ ]:
adata.obs.sample(n=5)

In [ ]:
print("# cells, # genes before filtering:", adata.shape)

sc.pp.filter_genes(adata, min_counts=3)
sc.pp.filter_cells(adata, min_counts=3)

print("# cells, # genes after filtering:", adata.shape)

In [ ]:
adata.layers["counts"] = adata.X.copy()

In [ ]:
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)

In [ ]:
adata.raw = adata

In [ ]:
mdata = muon.MuData({"rna": adata.copy(), "log_norm_rna": adata.copy()}, axis=-1)
# Now rna is count-based and log_norm_rna is log-normalized
mdata.mod["rna"].X = mdata.mod["rna"].layers["counts"]
del mdata.mod["rna"].raw
del mdata.mod["rna"].layers["counts"]
del mdata.mod["log_norm_rna"].layers["counts"]
mdata

In [ ]:
scvi.model.SCVI.setup_anndata(adata, layer="counts", batch_key="batch")

In [ ]:
np.asarray(pbmc5k.mod["prot"].X)

In [ ]:
# totalVI requires dense protein data
pbmc5k.mod["prot"].X = np.asarray(pbmc5k.mod["prot"].X.A)
scvi.model.TOTALVI.setup_mudata(
    pbmc5k,
    protein_layer=None,
    rna_layer=None,
    modalities={"protein_layer": "prot", "rna_layer": "rna"},
)

In [ ]:
# Train the model
scvi.model.SCVI.setup_anndata(adata)
vae = scvi.model.SCVI(adata)
vae.train()

solo = scvi.external.SOLO.from_scvi_model(vae)
solo.train()

# See if we have doublets
doublets = solo.predict()
doublets["prediction"] = solo.predict(soft=False)

# Strip off the "-1" which is on the barcodes
doublets.index = doublets.index.map(lambda x: x[:-2])

if verbosity == True:
	display(doublets)

# Count the number of doublets
display(doublets.groupby("prediction").count())

# Create a doublet "difference" score parameter in `df.["DSS"]`
doublets["DSS"] = doublets["doublet"] - doublets["singlet"]
doublets
if verbosity == True:
	display(doublets)

In [ ]:
sample_name = "Heart-Subsampled"

# Create a new column to contain a cell barcode starting with the sample name
adata.obs["Cell_Barcode"] = sample_name
# Append the index (cell barcode) to the sample name in each row
adata.obs['Cell_Barcode'] = adata.obs['Cell_Barcode'].map(str) + "_" + adata.obs.index

# Strip off the "-1" which is on the barcodes
adata.obs['Cell_Barcode'] = adata.obs['Cell_Barcode'].map(lambda x: x[:-2])

# Confirm the number of unique barcodes (should equal the number of rows)
display(f"All `adata.obs` rows have a unique barcode: {len(adata.obs['Cell_Barcode'].unique()) == adata.obs.shape[0]} ({len(adata.obs['Cell_Barcode'].unique())} cells barcoded)")

# Create a new column to contain a cell barcode starting with the sample name
doublets['Cell_Barcode'] = sample_name

# Append the index (cell barcode) to the sample name in each row
doublets['Cell_Barcode'] = doublets['Cell_Barcode'].map(str) + "_" + doublets.index


# Confirm the number of unique barcodes (should equal the number of rows)
display(f"All `doublets` rows have a unique barcode: {len(doublets['Cell_Barcode'].unique()) == doublets.shape[0]} ({len(doublets['Cell_Barcode'].unique())} cells barcoded)")

# Confirm that the doublets dataframe has the same barcodes as the adata.obs dataframe
display(f"Do adata.obs and doublets have the same barcodes?\n{doublets['Cell_Barcode'].isin(adata.obs['Cell_Barcode']).value_counts()}")

# Merge the doublets dataframe into adata.obs
adata.obs = pd.merge(adata.obs, doublets, on='Cell_Barcode')

# Make the cell barcodes be the index column
adata.obs.set_index('Cell_Barcode')

In [ ]:
# Basic quality control
sc.pp.filter_genes(adata, min_cells=3)
sc.pp.filter_cells(adata, min_genes=3000)

# Note this is an incomplete set of QC. We are proving that scanpy is operational

In [ ]:
# Normalize and Log Transform
adata.layers["counts"] = adata.X.copy()
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
adata.raw = adata

In [ ]:
sc.pp.highly_variable_genes(
    adata,
    n_top_genes=1200,
    subset=True,
    layer="counts",
    flavor="seurat_v3",
    batch_key="cell_source",
)